In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [148]:
train_data = pd.read_json('train_file.json')
val_data = pd.read_json('val_file.json')

In [149]:
print(len(train_data))
print(len(val_data))

6740
843


In [150]:
train_data.head()

,episode,speakers,emotions,utterances,triggers
0,utterance_3492,"[Phoebe, Eric, Phoebe, Eric, Phoebe]","[surprise, fear, surprise, sadness, disgust]","[You-you you had sex with Ursula?!, Uh, a litt...","[1.0, 1.0, 0.0, 0.0, 0.0]"
1,utterance_3952,"[Monica, Monica, Phoebe, Joey, Joey, Joey, Rac...","[disgust, disgust, anger, sadness, surprise, a...","[Dad, please don't pick your teeth out here!, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,utterance_3198,"[Older Scientist, Ross, Ross, Joey, Ross, Ross...","[neutral, neutral, neutral, neutral, neutral, ...","[Dr. Geller, there's a seat over here., Thank ...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0]"
3,utterance_2834,"[Monica, Monica, Monica]","[neutral, surprise, neutral]","[So, how'd the lasagne go over?, Really?!, Good.]","[0.0, 0.0, 1.0]"
4,utterance_453,"[Kate, The Director, Kate]","[joy, sadness, sadness]","[Become a drama critic!, I am hurt! A plague ...","[0.0, 0.0, 1.0]"


In [151]:
#join utterances with @ sign
train_data['utterances'] = train_data['utterances'].apply(lambda x: '@'.join(x))
train_data['speakers'] = train_data['speakers'].apply(lambda x: '@'.join(x))
val_data['utterances'] = val_data['utterances'].apply(lambda x: '@'.join(x))
val_data['speakers'] = val_data['speakers'].apply(lambda x: '@'.join(x))

In [152]:
# dropping duplicates 
train_data = train_data.drop_duplicates(subset=['speakers', 'utterances'], keep='first')
val_data = val_data.drop_duplicates(subset=['speakers', 'utterances'], keep='first')

In [153]:
train_data['utterances'] = train_data['utterances'].apply(lambda x: x.split('@'))
train_data['speakers'] = train_data['speakers'].apply(lambda x: x.split('@'))
val_data['utterances'] = val_data['utterances'].apply(lambda x: x.split('@'))
val_data['speakers'] = val_data['speakers'].apply(lambda x: x.split('@'))

In [154]:
# resetting index as we dropped rows
train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)

In [155]:
# print((train_data['utterances'][0]))
n = len(train_data['utterances'][0])
for i in range(n):
    print((train_data['utterances'][0][i]))

You-youyou had sex with Ursula?!
Uh, a little bit. She-she-she walked in and I thought she was you and I kissed her and
You didn't notice she was wearing different clothes?!
Well I was just so excited to see you.
Oh. Ew! Ew! Ew! Ugh! Y'know what? This is too weird.


In [156]:
unicode_mapping = {}
# unicode_mapping['\u0085'] = '...' 
# unicode_mapping['\u0091'] = "'"
# unicode_mapping['\u0092'] = "'"
# unicode_mapping['\u0093'] = '"'
# unicode_mapping['\u0094'] = '"'
# unicode_mapping['\u0097'] = '--'

# unicode_mapping['\u2014'] = '--'
# unicode_mapping['\u2019'] = "'"
# unicode_mapping['\u2026'] = '...'

# unicode_mapping['\u00e9'] = 'e'

unicode_mapping['\x85'] = '...' 
unicode_mapping['\x91'] = "'"
unicode_mapping['\x92'] = "'"
unicode_mapping['\x93'] = '"'
unicode_mapping['\x94'] = '"'
unicode_mapping['\x97'] = '--'

unicode_mapping['\u2014'] = '--'
unicode_mapping['\u2019'] = "'"
unicode_mapping['\u2026'] = '...'

unicode_mapping['\xe9'] = 'e'



In [157]:
# replacing unicode characters in the data
def clean_utterance(utterance_list):
    '''	
    This function takes a list of utterances and replaces the unicode with the proper characters.
    '''
    cleaned_utterances_list = []
    for utterance in utterance_list:
        for key in unicode_mapping:
            utterance = utterance.replace(key, unicode_mapping[key])
        cleaned_utterances_list.append(utterance)
    return cleaned_utterances_list

# clean the train data
n_train = len(train_data['utterances'])
for i in train_data.index:
    temp = train_data.loc[i].copy()
    cleaned_utterances = clean_utterance(temp['utterances'])
    temp['utterances'] = cleaned_utterances
    train_data.loc[i] = temp

# clean the val data
n_val = len(val_data['utterances'])
for i in val_data.index:
    temp = val_data.loc[i].copy()
    cleaned_utterances = clean_utterance(temp['utterances'])
    temp['utterances'] = cleaned_utterances
    val_data.loc[i] = temp

In [158]:
print(train_data['utterances'][0])
# print(len(train_data['utterances'][0]))

['You-you...you had sex with Ursula?!', 'Uh, a little bit. She-she-she walked in and I thought she was you and I kissed her and', "You didn't notice she was wearing different clothes?!", 'Well I was just so excited to see you.', "Oh. Ew! Ew! Ew! Ugh! Y'know what? This is too weird."]


In [159]:
train_data.head()

,episode,speakers,emotions,utterances,triggers
0,utterance_3492,"[Phoebe, Eric, Phoebe, Eric, Phoebe]","[surprise, fear, surprise, sadness, disgust]","[You-you...you had sex with Ursula?!, Uh, a li...","[1.0, 1.0, 0.0, 0.0, 0.0]"
1,utterance_3952,"[Monica, Monica, Phoebe, Joey, Joey, Joey, Rac...","[disgust, disgust, anger, sadness, surprise, a...","[Dad, please don't pick your teeth out here!, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,utterance_3198,"[Older Scientist, Ross, Ross, Joey, Ross, Ross...","[neutral, neutral, neutral, neutral, neutral, ...","[Dr. Geller, there's a seat over here., Thank ...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0]"
3,utterance_2834,"[Monica, Monica, Monica]","[neutral, surprise, neutral]","[So, how'd the lasagne go over?, Really?!, Good.]","[0.0, 0.0, 1.0]"
4,utterance_453,"[Kate, The Director, Kate]","[joy, sadness, sadness]","[Become a drama critic!, I am hurt! A plague ...","[0.0, 0.0, 1.0]"


In [160]:
print(val_data.shape)

(808, 5)
